In [14]:
from logicOp import *

In [15]:
blocksize = 4
blockNum = 4
n = blocksize * blockNum
start = time.time()

In [16]:
U = getU(blocksize, blockNum)
P = U @ U.conj().T
Q = np.identity(2**n) - P
print(MEqual(P@P, P))

MemoryError: Unable to allocate 32.0 GiB for an array with shape (65536, 65536) and data type float64

In [4]:
g = 3
Xeff = (1,0) * (n//2)
Zeff = (g,0) * (n//2)
Hpen = getHamil(n, Xeff, Zeff)

In [5]:
HpenInverse = np.linalg.pinv(Hpen)

In [9]:
def EncodeTarBlock(Htar, blockNum, lamb ,crossTerm):
    logDict = {}
    for i in range(blockNum):
        logDict[f'X{2*i}'] = [(-1, f'X{4*i}*X{4*i+2}')]
        logDict[f'X{2*i+1}'] = [(1, f'X{4*i}*X{4*i+1}')]
        logDict[f'Z{2*i}'] = [(1, f'Z{4*i}*Z{4*i+1}')]
        logDict[f'Z{2*i+1}'] = [(1, f'Z{4*i}*Z{4*i+2}')]
        logDict[f'Z{2*i}*Z{2*i+1}'] = [(1, f'Z{4*i+1}*Z{4*i+2}')]
        # logDict[f'Z{2*i+1}*Z{2*i+2}'] = [(1, f'Z{4*i+4}*Z{4*i+5}'), ((8*lamb/3)**0.5, f'Z{4*i+1}*X{4*i+7}+Z{4*i+3}*X{4*i+7}')]
        logDict[f'Z{2*i+1}*Z{2*i+2}'] = crossTerm(i)
    res = []
    for term in Htar:
        after = logDict[term[1]]
        for item in after:
            res.append((item[0]*term[0], item[1]))
    return res

In [8]:
T = Hpen @ HpenInverse @ Hpen
# print(MEqual(T, Hpen))
lambdaPen = 16
Q0 = HpenInverse @ Hpen
P0 = np.identity(Q0.shape[0]) - Q0

In [10]:

def crossTerm(i):
    return [(1, f'Z{4*i+1}*X{4*i+7}+Z{4*i+3}*X{4*i+7}')]
Henc_block = getHencBlock(blockNum, lambdaPen=16, crossTerm=crossTerm)
Htar_block = getHtarBlock(blockNum)
Henc_block = EncodeTarBlock(Htar_block, blockNum, lambdaPen, crossTerm)
Henc = blocks2Mat(n, Henc_block)
Htar = blocks2Mat(n//2, Htar_block)

In [11]:
Heff_2nd_order = P0@Henc@P0 - (P0@Henc@Q0@HpenInverse@Q0@Henc@P0 / lambdaPen)

In [12]:
print(checkSame(P @ Heff_2nd_order @ P, U@Htar@U.conj().T))

False


In [13]:
print(checkSame(P @ Heff_2nd_order @ (P0-P), np.zeros(P.shape)))

False
